In [3]:
import pandas as pd
import numpy as np
import folium
import time

In [2]:
fields = ["state", "parent_metro_region", 'period_end', 'region', 'property_type', 'period_end', 'new_listings' ,
          'median_sale_price', 'homes_sold','off_market_in_two_weeks', 'median_sale_price','median_dom','months_of_supply']

# read us_national_market_tracker.tsv000  into df
# df = pd.read_csv("zip_code_market_tracker.tsv000", delimiter = "\t", usecols = fields)

In [4]:
url = 'https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/zip_code_market_tracker.tsv000.gz'

start = time.time()
df = pd.read_csv(url, compression = 'gzip', sep='\t', on_bad_lines='skip')
# end time
execution = (time.time()-start)
print('Execution time in minutes: ' + str(round(execution / 60, 2)))
print('Num of rows: ', len(df))
print('Num of columns: ', len(df.columns))
df.head()


Execution time in minutes: 9.63
Num of rows:  7345590
Num of columns:  58


,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2020-10-01,2020-12-31,90,zip code,2,38608,f,Zip Code: 93205,NaN,California,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,"Bakersfield, CA",12540,2023-09-03 14:29:21
1,2021-07-01,2021-09-30,90,zip code,2,34919,f,Zip Code: 80543,NaN,Colorado,...,0.535714,NaN,NaN,NaN,NaN,NaN,NaN,"Greeley, CO",24540,2023-09-03 14:29:21
2,2020-08-01,2020-10-31,90,zip code,2,38610,f,Zip Code: 93207,NaN,California,...,0.111111,NaN,NaN,NaN,0.0,0.000000,0.000000,"Visalia, CA",47300,2023-09-03 14:29:21
3,2021-10-01,2021-12-31,90,zip code,2,20992,f,Zip Code: 49021,NaN,Michigan,...,-0.025210,NaN,NaN,NaN,0.2,-0.244444,-0.514286,"Lansing, MI",29620,2023-09-03 14:29:21
4,2016-04-01,2016-06-30,90,zip code,2,3079,f,Zip Code: 08518,NaN,New Jersey,...,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,"Camden, NJ",15804,2023-09-03 14:29:21


In [6]:
# Read Zıpcode - location data
xf = pd.read_csv("../Datas/US Zip Codes from 2013 Government Data.csv")

In [7]:
df['ZIP'] = df['region'].str.split(":").str[1]

In [8]:
df = df[df['property_type'] == "All Residential"]

In [9]:
# Convert 'period_end' column to datetime
df['period_end'] = pd.to_datetime(df['period_end'])


# Create year column
df['year'] = df['period_end'].dt.year

# filter for 2023
df = df[df['year'] == 2023]

In [10]:
# create month column
df['month'] = df['period_end'].dt.month

In [11]:
# Find the index of the last record for each ZIP code
latest_month_idx = df.groupby('ZIP')['month'].idxmax()

# Filter the original DataFrame using the index of the last records
latest_records_df = df.loc[latest_month_idx]

In [12]:
latest_records_df['ZIP'] = latest_records_df['ZIP'].astype(int)
fin = latest_records_df.merge(xf, on="ZIP")

In [14]:
fin.to_csv("../Datas/data_for_first_tab.csv",index=False)